# Run Models Notebook

This notebook demonstrates executing code and uploading results to GitHub from Google Cloud Run.

In [ ]:
import os
import requests
import base64
import json
from datetime import datetime
from google.cloud import secretmanager

In [ ]:
# Get the GitHub token from Secret Manager
def get_github_token():
    try:
        client = secretmanager.SecretManagerServiceClient()
        name = f"projects/{os.environ.get('GOOGLE_CLOUD_PROJECT')}/secrets/github-token/versions/latest"
        response = client.access_secret_version(request={"name": name})
        return response.payload.data.decode("UTF-8")
    except Exception as e:
        print(f"Error getting GitHub token: {e}")
        return None
GITHUB_TOKEN = get_github_token()
print("Fetching GitHub token using project:", os.environ.get('GOOGLE_CLOUD_PROJECT'))
if not GITHUB_TOKEN:
    print("GITHUB_TOKEN not retrieved or is empty.")
else:
    print(f"GITHUB_TOKEN retrieved: {GITHUB_TOKEN[:5]}... (length={len(GITHUB_TOKEN)})")

In [ ]:
def upload_reports_to_github(repo, token, branch='main', commit_message='Report from notebook execution'):
    if not token:
        print("No GitHub token available")
        return

    # Create a simple report file
    report_content = f"""
# Execution Report

Generated: {datetime.now().isoformat()}
Environment: Google Cloud Run
Project: {os.environ.get('GOOGLE_CLOUD_PROJECT', 'unknown')}
Status: Success
"""

    headers = {
        'Authorization': f'token {token}',
        'Accept': 'application/vnd.github.v3+json'
    }

    file_path = f"reports/execution-{datetime.now().strftime('%Y%m%d-%H%M%S')}.md"
    content_encoded = base64.b64encode(report_content.encode('utf-8')).decode('utf-8')

    data = {
        'message': commit_message,
        'content': content_encoded,
        'branch': branch
    }

    url = f'https://api.github.com/repos/{repo}/contents/{file_path}'

    try:
        response = requests.put(url, headers=headers, json=data)

        if response.status_code == 201:
            result = response.json()
            print(f"Successfully uploaded report to GitHub!")
            print(f"File: {file_path}")
            return result
        else:
            print(f"Failed to upload. Status: {response.status_code}")
            return None

    except Exception as e:
        print(f"Error uploading to GitHub: {e}")
        return None

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(6, 4))
plt.plot([1, 2, 3], [1, 4, 9])
plt.title("Test Plot")
plt.savefig("output/test-plot.png")
print("Saved PNG: test-plot.png")


In [ ]:
from fpdf import FPDF

pdf = FPDF()
pdf.add_page()
pdf.set_font("Arial", size=12)
pdf.cell(200, 10, txt="Hello, this is a test PDF report.", ln=True)
pdf.output("output/test-report.pdf")
print("Saved PDF: test-report.pdf")


In [ ]:
html_content = "<h1>Sample HTML</h1><p>This was generated from a notebook cell.</p>"
with open("output/test-page.html", "w") as f:
    f.write(html_content)
print("Saved HTML: test-page.html")


In [ ]:
import pandas as pd

df = pd.DataFrame({
    "A": [1, 2, 3],
    "B": ["x", "y", "z"]
})
df.to_csv("output/test-data.csv", index=False)
print("Saved CSV: test-data.csv")


In [ ]:
print("Starting notebook execution...")


In [ ]:
import sys

TARGET_REPO = "AbhinavSivanandhan/reports"
print("=== GITHUB UPLOAD STAGE STARTED ===", file=sys.stderr)
if GITHUB_TOKEN:
    print("Uploading results to GitHub...", file=sys.stderr)
    print("Uploading to GitHub repo:", TARGET_REPO, file=sys.stderr)
    print("Using commit message: Automated report from Cloud Run", file=sys.stderr)
    print("GitHub token length:", len(GITHUB_TOKEN), file=sys.stderr)

    try:
        result = upload_reports_to_github(
            repo=TARGET_REPO,
            token=GITHUB_TOKEN,
            branch='main',
            commit_message='Automated report from Cloud Run'
        )

        if result:
            print("Upload result (truncated):", str(result)[:300], file=sys.stderr)
        else:
            print("Notebook executed but upload failed. No result returned.", file=sys.stderr)
    except Exception as e:
        print("Exception during GitHub upload:", e, file=sys.stderr)
else:
    print("No GitHub token available - skipping upload", file=sys.stderr)

print("=== NOTEBOOK COMPLETED ===", file=sys.stderr)
